In [38]:
import numpy as np
import pandas as pd
import re

In [53]:
tcof = pd.read_csv('E:\Corpus\TCOF_CNRTL\TCOF_TOTAL2304.csv', sep=',')
clapi = pd.read_csv('E:\Corpus\CLAPI_scrape\CLAPI_TOTAL.csv', sep=',')
cfpp = pd.read_csv('E:\Corpus\CFPP\CFPP_TOTAL0205.csv', sep=',')
crfp = pd.read_csv('E:\Corpus\CRFP_ORFEO\CRFP_TOTAL0405.csv', sep=',')

In [54]:
all_ = pd.concat([tcof, clapi, cfpp, crfp])

In [55]:
for x in list(set(crfp['date_enregistrement'].tolist())):
    print(x, ':', date(x))

00/01/1999 : 1999-01-00
00/12/1999 : 1999-12-00
00/02/2001 : 2001-02-00
00/11/1999 : 1999-11-00
20/05/2000 : 2000-05-20
00/00/2000 : 2000-00-00
05/11/1999 : 1999-11-05
00/01/2001 : 2001-01-00
04/11/1999 : 1999-11-04
00/01/2000 : 2000-01-00
00/00/1999 : 1999-00-00
00/12/2000 : 2000-12-00
00/04/2000 : 2000-04-00
00/04/2002 : 2002-04-00
00/02/1998 : 1998-02-00
00/11/2000 : 2000-11-00


In [56]:
def age(string):
    if isinstance(string, str):
        if re.match(r'\s*[0-9][0-9]? ans?|\s*[0-9][0-9]?', string):
            string = string.replace(' ans','')
        elif re.match(r'entre [0-9][0-9]? ans?  et [0-9][0-9]? ans?', string):
            pattern = r'entre ([0-9][0-9]?) ans?  et ([0-9][0-9]?) ans?'
            match = re.search(pattern, string)
            string = match.group(1) + '-' + match.group(2)
        elif re.match(r'[0-9][0-9]? - [0-9][0-9]?|[0-9][0-9]?-[0-9][0-9]?', string):
            pass
        elif re.match(r'entre  et [0-9][0-9]?[0-9]? ans?', string):
            string = 'NA'
        elif re.match(r"([0-9][0-9]?) ans? ([0-9][0-9]?) mois?", string):
            pattern = r'([0-9][0-9]?) ans? ([0-9][0-9]?) mois?'
            match = re.search(pattern, string)
            string = match.group(1) + ',' + match.group(2)
        else:
            string = 'NA'
    else:
        string = 'NA'
    return string
    

In [70]:
def sexe(string):
    try : 
        string = string.replace(' ','')
    except : 
        pass
    if isinstance(string, str):
        if string == 'M' or string =='Homme'or string =='homme':
            string = 'Homme'
        elif string == 'F' or string =='Femme' or string =='femme':
            string='Femme'
        else:
            string='NA'
    else:
        string = 'NA'
    return string

In [71]:
def date(string):
    month_dict = {
    'Jan': '01','Fev': '02','Mar': '03','Avr': '04','Mai': '05','Jui': '06',
    'Juil': '07','Aou': '08','Sep': '09','Oct': '10','Nov': '11', 'Dec': '12',
    'janvier': '01','février': '02','mars': '03','avril': '04','mai': '05','juin': '06',
    'juillet': '07','août': '08','septembre': '09','octobre': '10','novembre': '11', 'decembre': '12'}
    if isinstance(string, str): 
        if re.match(r'[0-9]{4}-[0-9]{2}-[0-9]{2}', string):
            pass
        elif re.match(r'([0-9][0-9]?) ([a-zA-Z]{3}) ([0-9]{4})', string):
            pattern = r'([0-9][0-9]?) ([a-zA-Z]{3}) ([0-9]{4})'
            match = re.search(pattern, string)
            string = match.group(3)+'-'+month_dict['{}'.format(match.group(2))]+'-'+match.group(1)
        elif re.match(r'([0-9]*[0-9]*).(\S*).([0-9]{4})',string):
            pattern = r'([0-9]*[0-9]*).(\S*).([0-9]{4})'
            match = re.search(pattern, string)
            try : 
                string = match.group(3) + '-' + month_dict['{}'.format(match.group(2))] + '-' + match.group(1) 
            except : 
                string = match.group(3) + '-' + match.group(2) + '-' + match.group(1) 
        else:
            string='NA'
    else:
        string = 'NA'
    return string

In [72]:
def native(string):
    if string=='nonnative':
        string = 'non natif'
    elif string=='native':
        string = 'natif'
    else:
        string='NA'
    return string

In [73]:
def nom_locuteur(string):
    if string == "(None, None)" or isinstance(string, float):
        return "NA"
    elif re.match(r'.*(\*\*\*NO SPEAKER\*\*\*).*', string):
        pattern = r'.*(\*\*\*NO SPEAKER\*\*\*).*'
        match = re.search(pattern, string)
        return match.group(1)
    else:
        return string

In [74]:
all_.columns

Index(['startTime', 'endTime', 'speaker', 'speaker_name', 'speaker_native',
       'text', 'text_synchronisé', 'encoding', 'sous_corpus', 'sexe_speaker',
       'age_speaker', 'profession_speaker', 'appartenance_speaker',
       'autre_langue_speaker', 'etude_speaker', 'description', 'duree',
       'responsable', 'date_enregistrement', 'langue_enregistrement',
       'lieu_enregistrement', 'path_trs', 'path_wav', 'corpus'],
      dtype='object')

In [75]:
def time2seconds(string):
    try : 
        pattern = r'([0-9]*)’([0-9]*)’’'
        match = re.search(pattern, string)
        if match : 
            liste = [0,int(match.group(1)),int(match.group(2))]
        else: 
            return string
        ftr = [3600,60,1]
        seconds = sum([multi*value for multi,value in zip(ftr, liste)])
        return seconds
    except: 
        return string


In [76]:
def normalize_df(df):
    cols = ['sous_corpus','description',  'duree',  'date_enregistrement',  'langue_enregistrement',
 'lieu_enregistrement', 'corpus','startTime', 'endTime', 'speaker', 'text',
 'text_synchronisé', 'speaker_name', 'speaker_native', 'sexe_speaker', 'age_speaker',
 'profession_speaker','appartenance_speaker', 'autre_langue_speaker', 'etude_speaker', 'encoding', 
            'responsable', 'path_trs', 'path_wav']
    df['age_speaker'] = df['age_speaker'].apply(lambda x : age(x))
    df['sexe_speaker'] = df['sexe_speaker'].apply(lambda x : sexe(x))
    df['date_enregistrement'] = df['date_enregistrement'].apply(lambda x : date(x))
    df['speaker_native'] = df['speaker_native'].apply(lambda x : native(x))
    #df['duree'] = df['duree'].apply(lambda x : time2seconds(x))
    df = df.replace('NaN', 'NA')
    df = df[cols]
    df = df.rename(columns={'sous_corpus': 'enregistrement', 'description': 'description_enregistrement',
                           'duree':'duree_enregistrement', 'startTime':'debut_tour', 'endTime':'fin_tour',
                           'speaker':'locuteur','text':'segment','text_synchronisé':'segment_synchronise',
                           'speaker_name':'nom_locuteur', 'speaker_native':'statut_linguistique_locuteur',
                           'sexe_speaker':'sexe_locuteur','appartenance_speaker':'appartenance_locuteur',
                            'age_speaker':'age_locuteur','autre_langue_speaker':'autre_langue_locuteur','etude_speaker':'etude_locuteur',
                           'profession_speaker':'profession_locuteur'})
    df['nom_locuteur'] = df['nom_locuteur'].apply(lambda x : nom_locuteur(x))
    df['locuteur'] = df['locuteur'].apply(lambda x : nom_locuteur(x))
    df['profession_locuteur'] = df['profession_locuteur'].apply(lambda x : nom_locuteur(x))
    df = df.drop('responsable', axis=1)
    df = df.drop('encoding', axis=1)

    return df

In [77]:
all_ = normalize_df(all_)

In [78]:
all_

,enregistrement,description_enregistrement,duree_enregistrement,date_enregistrement,langue_enregistrement,lieu_enregistrement,corpus,debut_tour,fin_tour,locuteur,...,nom_locuteur,statut_linguistique_locuteur,sexe_locuteur,age_locuteur,profession_locuteur,appartenance_locuteur,autre_langue_locuteur,etude_locuteur,path_trs,path_wav
0,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,0.000000,35.592000,spk1,...,L1,non natif,Homme,NA,NA,NaN,NaN,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
1,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,35.592000,35.832000,spk2,...,L2,natif,Femme,NA,NA,NaN,NaN,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
2,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,35.832000,127.303000,spk1,...,L1,non natif,Homme,NA,NA,NaN,NaN,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
3,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,127.303000,127.569000,spk2,...,L2,natif,Femme,NA,NA,NaN,NaN,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
4,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,127.569000,142.713000,spk1,...,L1,non natif,Homme,NA,NA,NaN,NaN,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41795,PUB-VAL-1,trois interventions téléphoniques : débat sur...,904.000,2000-00-00,français,"France, Rhône-Alpes, Valence",CRFP,897.750000,899.869995,L4,...,NA,NA,Homme,NA,inconnu,inconnu,NaN,inconnu,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.orfeo,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.wav
41796,PUB-VAL-1,trois interventions téléphoniques : débat sur...,904.000,2000-00-00,français,"France, Rhône-Alpes, Valence",CRFP,898.429993,899.409973,L1,...,NA,NA,Homme,NA,inconnu,inconnu,NaN,inconnu,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.orfeo,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.wav
41797,PUB-VAL-1,trois interventions téléphoniques : débat sur...,904.000,2000-00-00,français,"France, Rhône-Alpes, Valence",CRFP,900.359985,902.880005,L1,...,NA,NA,Homme,NA,inconnu,inconnu,NaN,inconnu,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.orfeo,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.wav
41798,PUB-VAL-1,trois interventions téléphoniques : débat sur...,904.000,2000-00-00,français,"France, Rhône-Alpes, Valence",CRFP,901.539978,902.590027,L4,...,NA,NA,Homme,NA,inconnu,inconnu,NaN,inconnu,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.orfeo,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.wav


In [80]:
enregistrements = list(set(all_['enregistrement'].tolist()))
result = [all_.loc[all_['enregistrement'] == e].iloc[0] for e in enregistrements]
duree_totale = np.nansum([x['duree_enregistrement'] for x in result])

In [81]:
len(enregistrements)

766

In [82]:
duree_totale

756265.779

In [83]:
all_.fillna("NA")

,enregistrement,description_enregistrement,duree_enregistrement,date_enregistrement,langue_enregistrement,lieu_enregistrement,corpus,debut_tour,fin_tour,locuteur,...,nom_locuteur,statut_linguistique_locuteur,sexe_locuteur,age_locuteur,profession_locuteur,appartenance_locuteur,autre_langue_locuteur,etude_locuteur,path_trs,path_wav
0,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,0.000000,35.592000,spk1,...,L1,non natif,Homme,NA,NA,NA,NA,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
1,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,35.592000,35.832000,spk2,...,L2,natif,Femme,NA,NA,NA,NA,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
2,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,35.832000,127.303000,spk1,...,L1,non natif,Homme,NA,NA,NA,NA,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
3,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,127.303000,127.569000,spk2,...,L2,natif,Femme,NA,NA,NA,NA,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
4,theorielinguistique_07,Entretien Discussions sur la communication ver...,897.971,NA,français,,TCOF,127.569000,142.713000,spk1,...,L1,non natif,Homme,NA,NA,NA,NA,Inconnu,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...,E:\Corpus\TCOF_CNRTL\theorielinguistique_07\th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41795,PUB-VAL-1,trois interventions téléphoniques : débat sur...,904.0,2000-00-00,français,"France, Rhône-Alpes, Valence",CRFP,897.750000,899.869995,L4,...,NA,NA,Homme,NA,inconnu,inconnu,NA,inconnu,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.orfeo,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.wav
41796,PUB-VAL-1,trois interventions téléphoniques : débat sur...,904.0,2000-00-00,français,"France, Rhône-Alpes, Valence",CRFP,898.429993,899.409973,L1,...,NA,NA,Homme,NA,inconnu,inconnu,NA,inconnu,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.orfeo,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.wav
41797,PUB-VAL-1,trois interventions téléphoniques : débat sur...,904.0,2000-00-00,français,"France, Rhône-Alpes, Valence",CRFP,900.359985,902.880005,L1,...,NA,NA,Homme,NA,inconnu,inconnu,NA,inconnu,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.orfeo,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.wav
41798,PUB-VAL-1,trois interventions téléphoniques : débat sur...,904.0,2000-00-00,français,"France, Rhône-Alpes, Valence",CRFP,901.539978,902.590027,L4,...,NA,NA,Homme,NA,inconnu,inconnu,NA,inconnu,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.orfeo,E:/Corpus/CRFP_ORFEO/PUB-VAL-1.wav


In [85]:
all_.to_csv('E:\Corpus\TOTAL_0405.csv', sep=',', encoding='utf-8', index=False)